# Groundedness Checking Skills

In this notebook we run a simple grounding pipeline, to see if a summary text has any ungrounded additions as compared to the original.

Let us first define our grounding text:

In [ ]:
grounding_text = """I am by birth a Genevese, and my family is one of the most distinguished of that republic.
My ancestors had been for many years counsellors and syndics, and my father had filled several public situations
with honour and reputation. He was respected by all who knew him for his integrity and indefatigable attention
to public business. He passed his younger days perpetually occupied by the affairs of his country; a variety
of circumstances had prevented his marrying early, nor was it until the decline of life that he became a husband
and the father of a family.

As the circumstances of his marriage illustrate his character, I cannot refrain from relating them. One of his
most intimate friends was a merchant who, from a flourishing state, fell, through numerous mischances, into poverty.
This man, whose name was Beaufort, was of a proud and unbending disposition and could not bear to live in poverty
and oblivion in the same country where he had formerly been distinguished for his rank and magnificence. Having
paid his debts, therefore, in the most honourable manner, he retreated with his daughter to the town of Lucerne,
where he lived unknown and in wretchedness. My father loved Beaufort with the truest friendship and was deeply
grieved by his retreat in these unfortunate circumstances. He bitterly deplored the false pride which led his friend
to a conduct so little worthy of the affection that united them. He lost no time in endeavouring to seek him out,
with the hope of persuading him to begin the world again through his credit and assistance.

Beaufort had taken effectual measures to conceal himself, and it was ten months before my father discovered his
abode. Overjoyed at this discovery, he hastened to the house, which was situated in a mean street near the Reuss.
But when he entered, misery and despair alone welcomed him. Beaufort had saved but a very small sum of money from
the wreck of his fortunes, but it was sufficient to provide him with sustenance for some months, and in the meantime
he hoped to procure some respectable employment in a merchant's house. The interval was, consequently, spent in
inaction; his grief only became more deep and rankling when he had leisure for reflection, and at length it took
so fast hold of his mind that at the end of three months he lay on a bed of sickness, incapable of any exertion.

His daughter attended him with the greatest tenderness, but she saw with despair that their little fund was
rapidly decreasing and that there was no other prospect of support. But Caroline Beaufort possessed a mind of an
uncommon mould, and her courage rose to support her in her adversity. She procured plain work; she plaited straw
and by various means contrived to earn a pittance scarcely sufficient to support life.

Several months passed in this manner. Her father grew worse; her time was more entirely occupied in attending him;
her means of subsistence decreased; and in the tenth month her father died in her arms, leaving her an orphan and
a beggar. This last blow overcame her, and she knelt by Beaufort's coffin weeping bitterly, when my father entered
the chamber. He came like a protecting spirit to the poor girl, who committed herself to his care; and after the
interment of his friend he conducted her to Geneva and placed her under the protection of a relation. Two years
after this event Caroline became his wife."""

## Set up Semantic Kernel

We prepare our kernel in the usual way:

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

## Import the Skills

We are going to be using two Skills: the `SummarizeSkill` to produce a summary of the above text, and then the grounding skill, to check its quality, and remove ungrounded additions:

In [ ]:
from semantic_kernel.core_skills.text_skill import TextSkill

# note: using skills from the samples folder
skills_directory = "../../skills"

groundingSemanticFunctions = kernel.import_semantic_skill_from_directory(skills_directory, "GroundingSkill")
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")

We can also extract the individual semantic functions for our use:

In [ ]:
create_summary = summarize_skill["Summarize"]
entity_extraction = groundingSemanticFunctions["ExtractEntities"]
reference_check = groundingSemanticFunctions["ReferenceCheckEntities"]
entity_excision = groundingSemanticFunctions["ExciseEntities"]

## Calling Individual Semantic Functions

We will start by calling the individual grounding functions in turn, to show their use. For this we need to create a same summary text:

In [ ]:
summary_text = """
My father, a respected resident of Milan, was a close friend of a merchant named Beaufort who, after a series of
misfortunes, moved to Zurich in poverty. My father was upset by his friend's troubles and sought him out,
finding him in a mean street. Beaufort had saved a small sum of money, but it was not enough to support him and
his daughter, Mary. Mary procured work to eek out a living, but after ten months her father died, leaving
her a beggar. My father came to her aid and two years later they married.
"""

summary_text = summary_text.replace("\n", " ").replace("  ", " ")
print(summary_text)

Some things to note:

- The implied residence of Geneva has been changed to Milan
- Lucerne has been changed to Zurich
- Caroline has been renamed as Mary


The grounding skill has three stages:

1. Extract entities from a summary text
2. Perform a reference check against the grounding text
3. Excise any entities which failed the reference check from the summary

Now, let us start calling individual semantic functions.

### Preparing the Context

Semantic functions operate in a context, which provides extra parameters for their operation. For the grounding skill, the context is expected to supply the topic for the reference checking, and some particular examples of them:

In [ ]:
context = kernel.create_new_context()
context["topic"] = "people and places"
context["example_entities"] = "John, Jane, mother, brother, Paris, Rome"

### Extracting the Entities

The first function we need is entity extraction. We are going to take our summary text, and get a list of entities found within it. For this we use `entity_extraction()`:

In [ ]:
extraction_result = entity_extraction(summary_text, context=context)

print(extraction_result)

We see that the output consists of two parts: the list of entities, and a repeat of the summary.

### Performing the reference check

We now use the grounding text to see if the entities we found are grounded. We start by adding the grounding text to our context:

In [ ]:
context["reference_context"] = grounding_text

With this in place, we can run the reference checking function:

In [ ]:
grounding_result = reference_check(extraction_result.result, context=context)

print(grounding_result.result)

So we now have a list of ungrounded entities (of course, this list may not be well grounded itself), and a repeat of the summary text. The observant reader may be wondering why this is being dragged along; all will soon be revealed.

### Excising the ungrounded entities

Finally we can remove the ungrounded entities from the text. For this, we don't need to change the context again - the output of the reference check contains all that we need:

In [ ]:
excision_result = entity_excision(grounding_result.result)

print(excision_result.result)

## Using a Planner

Calling individual semantic kernel functions in the manner described above has been extensively focus-group tested, and declared to be 'statistically not fun.' Fortunately, _Planners_ are provided which can take a set of functions, and figure out a way to combine them to perform a task.

So in our case, suppose that we want to take the above original text, produce a summary and then check it for groundedness. We might write an instruction such as:

In [ ]:
target_topic = "people and places"
samples = "John, Jane, mother, brother, Paris, Rome"

ask = f"""Make a summary of the following [ORIGINAL_TEXT]. Then make a list of entities
related to {target_topic} (such as {samples}) which are present in the summary.
Take this list of entities, and from it make another list of those which are not
grounded in the [ORIGINAL_TEXT]. Finally, rewrite your summary to remove the entites
which are not grounded in the original.

This is the original text:

[ORIGINAL_TEXT]
{grounding_text}
[/ORIGINAL_TEXT]
"""

print(ask)

Next, we set up a planner prompt, which can guide the planner towards the result we desire:

In [ ]:
PROMPT = """
# Task

1. You are a planner for the Semantic Kernel.
2. Your job is to create a properly formatted JSON plan step by step, to satisfy the goal given.
3. Create a list of subtasks based off the [GOAL] provided.
4. Each subtask must be from within the [AVAILABLE_FUNCTIONS] list. Do not use any functions that are not in the list.
5. Base your decisions on which functions to use from the description and the name of the function.
6. Sometimes, a function may take arguments. Provide them if necessary.
7. The plan should be as short as possible.
8. The plan does not have to use all of the [AVAILABLE_FUNCTIONS]
9. Ensure that the plan you project is valid JSON. Remove new line characters if necessary

Here are some examples:

## Example 1

[AVAILABLE_FUNCTIONS]
EmailConnector.LookupContactEmail
description: looks up the a contact and retrieves their email address
args:
- name: the name to look up

WriterSkill.EmailTo
description: email the input text to a recipient
args:
- input: the text to email
- recipient: the recipient's email address. Multiple addresses may be included if separated by ';'.

WriterSkill.Translate
description: translate the input to another language
args:
- input: the text to translate
- language: the language to translate to

WriterSkill.Summarize
description: summarize input text
args:
- input: the text to summarize

FunSkill.Joke
description: Generate a funny joke
args:
- input: the input to generate a joke about
[/AVAILABLE_FUNCTIONS]

[GOAL]
Tell a joke about cars. Translate it to Spanish
[/GOAL]

[OUTPUT]
    {
        "input": "cars",
        "subtasks": [
            {"function": "FunSkill.Joke"},
            {"function": "WriterSkill.Translate", "args": {"language": "Spanish"}}
        ]
    }

## Example 2

[AVAILABLE_FUNCTIONS]
WriterSkill.Brainstorm
description: Brainstorm ideas
args:
- input: the input to brainstorm about

WriterSkill.FollowStyle
description: Rewrite a text in a given style
args:
- input: the original text
- text_sample: A sample of text whose style should be used to rewrite the input

WriterSkill.EmailTo
description: Write an email to a recipient
args:
- input: the input to write about
- recipient: the recipient's email address.

WriterSkill.Translate
description: translate the input to another language
args:
- input: the text to translate
- language: the language to translate to
[/AVAILABLE_FUNCTIONS]

[GOAL]
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Edgar Allen Poe so write using his style.
E-mail these ideas to my significant other. Translate it to French.

The following is a sample text by Edgar Allen Poe:

[SAMPLE_TEXT]
Take thy beak from out my heart, and take thy form from off my door!
Quoth the Raven, 'Nevermore.'
[/SAMPLE_TEXT]
[/GOAL]

[OUTPUT]
    {
        "input": "Valentine's Day Date Ideas",
        "subtasks": [
            {"function": "WriterSkill.Brainstorm"},
            {"function": "WriterSkill.FollowStyle", "args" : { "text_sample": "Take thy beak from out my heart, and take thy form from off my door!\nQuoth the Raven, 'Nevermore.'"}},
            {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
            {"function": "WriterSkill.Translate", "args": {"language": "French"}}
        ]
    }
    
## Example 3
    
[AVAILABLE_FUNCTIONS]
TextSkill.Append
description: append one text after another
args:
- input: the first text
- second_text: the text to place after the first

WriterSkill.Translate
description: translate the input to another language
args:
- input: the text to translate
- language: the language to translate to
[/AVAILABLE_FUNCTIONS]

[GOAL]
I have two texts from the same original piece. The
first needs to be translated into English, and then concatenated
with the second.

This is the first text:
[TEXT_A]
GALLIA est omnis divisa in partes tres, quarum unam incolunt Belgae,
aliam Aquitani, tertiam qui ipsorum lingua Celtae, nostra Galli appellantur
[/TEXT_A]

This is the second text:
[TEXT_B]
All these differ from each other in language, customs and laws
[/TEXT_B]
[/GOAL]

[OUTPUT]
    {
        "input": "GALLIA est omnis divisa in partes tres, quarum unam incolunt Belgae,\naliam Aquitani, tertiam qui ipsorum lingua Celtae, nostra Galli appellantur",
        "subtasks": [
            {"function": "WriterSkill.Translate", "args": { "language": "english" }},
            {"function": "TextSkill.Append", "args" : { "second_text": "All these differ from each other in language, customs and laws"}}
        ]
    }

# Task

Produce an execution plan for the following goal, based on these functions:

[AVAILABLE_FUNCTIONS]
{{$available_functions}}
[/AVAILABLE_FUNCTIONS]

[GOAL]
{{$goal}}
[/GOAL]

[OUTPUT]
"""

Now to do the planning. For this, Semantic Kernel provides a `BasicPlanner` object:

In [ ]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

To avoid the possibility of cross contamination, we create a fresh kernel:

In [ ]:
kernel_2 = sk.Kernel()

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel_2.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel_2.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))
    
groundingSemanticFunctions = kernel_2.import_semantic_skill_from_directory(skills_directory, "GroundingSkill")
summarize_skill = kernel_2.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")

And with all that in place, we can generate the plan:

In [ ]:
my_plan = await planner.create_plan_async(ask, kernel_2, prompt=PROMPT)

print(my_plan.generated_plan)

With that in place, we can try executing the plan to get our grounded summary:

In [ ]:
results = await planner.execute_plan_async(my_plan, kernel_2, debug_print=True)

And the result:

In [ ]:
print(results)